In [ ]:

!pip -q install requests beautifulsoup4 pandas rapidfuzz pdfplumber camelot-py[cv] tabula-py
!apt-get -yqq install ghostscript

import os, re, io, json, time, tempfile, warnings
import requests
import pandas as pd
import pdfplumber
from bs4 import BeautifulSoup
from rapidfuzz import fuzz, process
from tqdm import tqdm
tqdm.pandas()

# Optional backends
import camelot
import tabula

warnings.filterwarnings("ignore")


# SOURCE URLS & CONFIG


In [ ]:
NLEM_URL = "https://cdsco.gov.in/opencms/resources/UploadCDSCOWeb/2018/UploadConsumer/nlem2022.pdf"
CDSCO_INDEX_URL = "https://cdsco.gov.in/opencms/opencms/en/Approval_new/Approved-New-Drugs/"

YEAR_RANGE = list(range(2018, 2026))

OUT_DIR = "/content/indian_meds_ref"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; ColabParser/1.0)"}

def dl(url, path):
    r = requests.get(url, headers=HEADERS, timeout=60)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)
    return path

# Basic text normalization for matching generic names
DOSAGE_WORDS = [
    r"\btablet(s)?\b", r"\bcapsule(s)?\b", r"\bsyrup\b", r"\bsuspension\b", r"\binjection\b",
    r"\bcream\b", r"\boxy?ment\b", r"\bdrops\b", r"\bsolution\b", r"\bgel\b", r"\blotion\b",
    r"\bprefilled pen\b", r"\bpre-filled pen\b", r"\bpenfill\b", r"\bpowder\b", r"\bpatch\b",
    r"\bdispersible\b", r"\bextended release\b", r"\bsustained release\b", r"\bER\b", r"\bSR\b"
]
DOSAGE_RE = re.compile("|".join(DOSAGE_WORDS), flags=re.I)
STRENGTH_RE = re.compile(r"\b\d+(?:\.\d+)?\s*(mg|mcg|g|ml|iu|%|\u00b5g)(/\d+\s*(ml|g))?\b", flags=re.I)
PAREN_RE = re.compile(r"[()\[\]{}]")
MULTISPACE_RE = re.compile(r"\s+")

SYNONYM_MAP = {
    # common global synonyms
    "acetaminophen": "paracetamol",
    "tylenol": "paracetamol",
    "ibuprofen lysine": "ibuprofen",
}


def normalize_name(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text.strip().lower()
    t = PAREN_RE.sub(" ", t)
    t = STRENGTH_RE.sub(" ", t)
    t = DOSAGE_RE.sub(" ", t)
    t = re.sub(r"[^a-z0-9 +\-/]", " ", t)
    t = MULTISPACE_RE.sub(" ", t).strip()
    # map synonyms
    t = SYNONYM_MAP.get(t, t)
    t = t.replace(" & ", " + ")
    return t

In [ ]:
def parse_tables_camelot(pdf_path):
    tables = []
    try:
        t_lattice = camelot.read_pdf(pdf_path, pages='all', flavor='lattice')
        tables.extend([t.df for t in t_lattice])
    except Exception:
        pass
    try:
        t_stream = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
        tables.extend([t.df for t in t_stream])
    except Exception:
        pass
    return tables


def parse_tables_tabula(pdf_path):
    try:
        dfs = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, guess=True, lattice=True)
        return dfs or []
    except Exception:
        return []


def extract_text_pdfplumber(pdf_path):
    chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            try:
                chunks.append(page.extract_text() or "")
            except Exception:
                chunks.append("")
    return "\n".join(chunks)

# VERIFY THE DATASET

In [ ]:

print("Downloading NLEM 2022…")
nlem_pdf = os.path.join(OUT_DIR, "nlem2022.pdf")
dl(NLEM_URL, nlem_pdf)

print("Parsing NLEM tables…")
frames = []
for parser in (parse_tables_camelot, parse_tables_tabula):
    tbls = parser(nlem_pdf)
    for df in tbls:
        # Try to find a column that looks like "Medicine" or similar
        cols = [c.strip().lower() for c in df.columns.astype(str)]
        df.columns = cols
        for key in ["medicine", "name of medicine", "drug", "medicine name", "generic name"]:
            if key in df.columns:
                sub = df[[key]].copy()
                sub.columns = ["medicine_raw"]
                frames.append(sub)
                break

if not frames:
    print("Falling back to text extraction for NLEM…")
    text = extract_text_pdfplumber(nlem_pdf)
    candidates = []
    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue
        if re.search(r"[A-Za-z]", line) and not line.lower().startswith(("section", "chapter")):
            candidates.append(line)
    nlem_df = pd.DataFrame({"medicine_raw": candidates})
else:
    nlem_df = pd.concat(frames, ignore_index=True)

nlem_df["medicine_clean"] = nlem_df["medicine_raw"].astype(str).apply(normalize_name)
# Filter out empty & obviously bad rows
nlem_df = nlem_df[nlem_df["medicine_clean"].str.len() > 1]
nlem_df = nlem_df.drop_duplicates("medicine_clean").reset_index(drop=True)
nlem_df["source"] = "NLEM2022"


print("Fetching CDSCO Approved New Drugs index…")
html = requests.get(CDSCO_INDEX_URL, headers=HEADERS, timeout=60).text
soup = BeautifulSoup(html, "html.parser")
links = []
for a in soup.find_all("a", href=True):
    href = a["href"].strip()
    if href.lower().endswith(".pdf") and ("new" in href.lower() or "drug" in href.lower()):
        links.append(href if href.startswith("http") else requests.compat.urljoin(CDSCO_INDEX_URL, href))

year_links = []
for u in links:
    if any(str(y) in u for y in YEAR_RANGE) or "1971_1980" in u or "1991" in u or "2019" in u:
        year_links.append(u)

year_links = sorted(set(year_links))
print(f"Found {len(year_links)} CDSCO PDFs to parse…")


cdsco_rows = []
for i, url in enumerate(year_links, 1):
    print(f"[{i}/{len(year_links)}] {url}")
    pdf_path = os.path.join(OUT_DIR, f"cdsco_{i}.pdf")
    try:
        dl(url, pdf_path)
    except Exception as e:
        print("  download failed:", e)
        continue

    found_tables = False
    for parser in (parse_tables_camelot, parse_tables_tabula):
        tbls = parser(pdf_path)
        if not tbls:
            continue
        for df in tbls:
            cols = [c.strip().lower() for c in df.columns.astype(str)]
            df.columns = cols
            # common header variants
            cand_cols = [
                "name of drug", "name of new drug", "drug name", "name of the drug", "name",
            ]
            for key in cand_cols:
                if key in df.columns:
                    sub = df[[key]].copy()
                    sub.columns = ["medicine_raw"]
                    sub["source_url"] = url
                    cdsco_rows.append(sub)
                    found_tables = True
                    break
        if found_tables:
            break

    if not found_tables:
        text = extract_text_pdfplumber(pdf_path)
        lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
        meds = []
        for ln in lines:
            if re.match(r"^\d+\.?\s+", ln):
                # drop serial; take rest up to two columns separated by two+ spaces
                rest = re.sub(r"^\d+\.?\s+", "", ln)
                # split on double spaces to approximate columns
                part = re.split(r"\s{2,}", rest)[0]
                meds.append(part)
        if meds:
            df_fallback = pd.DataFrame({"medicine_raw": meds})
            df_fallback["source_url"] = url
            cdsco_rows.append(df_fallback)

if cdsco_rows:
    cdsco_df = pd.concat(cdsco_rows, ignore_index=True)
    cdsco_df["medicine_clean"] = cdsco_df["medicine_raw"].astype(str).apply(normalize_name)
    cdsco_df = cdsco_df[cdsco_df["medicine_clean"].str.len() > 1]
    cdsco_df["source"] = "CDSCO_NewDrugs"
else:
    cdsco_df = pd.DataFrame(columns=["medicine_raw","medicine_clean","source","source_url"])


ref_df = pd.concat([
    nlem_df[["medicine_raw","medicine_clean","source"]],
    cdsco_df[["medicine_raw","medicine_clean","source"] + (["source_url"] if "source_url" in cdsco_df.columns else [])]
], ignore_index=True).dropna(subset=["medicine_clean"])\
 .drop_duplicates("medicine_clean").reset_index(drop=True)

ref_path = os.path.join(OUT_DIR, "indian_medicines_reference.csv")
ref_df.to_csv(ref_path, index=False)
print("\nSaved unified reference to:", ref_path)
print(ref_df.head())


DATASET_PATH = "/content/updated_indian_medicine_data.csv"
assert os.path.exists(DATASET_PATH), "Upload your medicine_data.csv to /content first."

user_df = pd.read_csv(DATASET_PATH)
# Choose what to match on: prefer salt/generic if available, else product name
candidate_col = "salt_composition" if "salt_composition" in user_df.columns else ("product_name" if "product_name" in user_df.columns else user_df.columns[0])
print("Matching on column:", candidate_col)

user_df["query_raw"] = user_df[candidate_col].astype(str)
user_df["query_clean"] = user_df["query_raw"].apply(normalize_name)

choices = ref_df["medicine_clean"].tolist()

from tqdm.auto import tqdm

def fuzzy_match_one(s, choices, threshold=90):
    if not isinstance(s, str) or not s.strip():
        return pd.Series({"match": None, "score": 0, "status": "No Query"})
    m = process.extractOne(s, choices, scorer=fuzz.ratio)
    if m is None:
        return pd.Series({"match": None, "score": 0, "status": "Not Found"})
    name, score, idx = m
    status = "✅ Found" if score >= threshold else ("⚠ Close Match" if score >= 75 else "❌ Not Found")
    return pd.Series({"match": name, "score": int(score), "status": status})

res = user_df["query_clean"].progress_apply(lambda x: fuzzy_match_one(x, choices))
report = pd.concat([user_df, res], axis=1)

report_path = os.path.join(OUT_DIR, "verification_report.csv")
report.to_csv(report_path, index=False)
print("Verification report saved to:", report_path)

# Quick summary
summary = report.groupby("status").size().reset_index(name="count").sort_values("count", ascending=False)
print("\nSummary:\n", summary)


Parsing NLEM tables…
Fetching CDSCO Approved New Drugs index…
Found 0 CDSCO PDFs to parse…

Saved unified reference to: /content/indian_meds_ref/indian_medicines_reference.csv
     medicine_raw  medicine_clean    source source_url
0  Glycopyrrolate  glycopyrrolate  NLEM2022        NaN
1      Midazolam*       midazolam  NLEM2022        NaN
2      Morphine**        morphine  NLEM2022        NaN
3             NaN             nan  NLEM2022        NaN
4        Medicine        medicine  NLEM2022        NaN
Matching on column: salt_composition


100%|██████████| 253973/253973 [00:53<00:00, 4775.54it/s]


Verification report saved to: /content/indian_meds_ref/verification_report.csv

Summary:
           status   count
1        ✅ Found  247628
2    ❌ Not Found    5616
0  ⚠ Close Match     729
